In [ ]:
from load_paus_cat import load_paus_cat

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo

In [ ]:
# Load PAUS cat
field_name = 'W3'
path_to_cat = f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_{field_name}.csv'
paus_cat = load_paus_cat(path_to_cat)

In [ ]:
# Load SDSS DR16Q catalog
sdss_dir = '/home/alberto/almacen/SDSS_spectra_fits'

path_to_sdss_cat = f'{sdss_dir}/DR16/DR16Q_Superset_v3.fits'
sdss_cat = fits.open(path_to_sdss_cat)[1].data

# And the line catalog
fread = fits.open(f'{sdss_dir}/spAllLine-v5_13_0.fits')
sdss_line_cat = fread[1].data[np.where(fread[1].data['LINEWAVE'] == 1215.67)]

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=paus_cat['RA'] * u.deg,
                       dec=paus_cat['DEC'] * u.deg)
coords_sdss = SkyCoord(ra=sdss_cat['RA'] * u.deg,
                       dec=sdss_cat['DEC'] * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_sdss)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

In [ ]:
F_lya = np.ones(len(paus_cat['RA'])).astype(float) * -1

for src in np.where(mask_dist)[0]:
    fiber = sdss_cat['FIBERID'][xm_id[src]]
    mjd = sdss_cat['MJD'][xm_id[src]]
    plate = sdss_cat['PLATE'][xm_id[src]]

    where_mjd = np.where(sdss_line_cat['MJD'] == mjd)
    where_mjd_pl = np.where(sdss_line_cat['PLATE'][where_mjd] == plate)
    where_mjd_pl_fi = np.where(sdss_line_cat['FIBERID'][where_mjd[0][where_mjd_pl]] == fiber)
    where_in_line_cat = where_mjd[0][where_mjd_pl[0][where_mjd_pl_fi]]

    if len(where_in_line_cat) > 0:
        F_lya[src] = sdss_line_cat['LINEAREA'][where_in_line_cat]
    else:
        print('nope')

In [ ]:
Dl = cosmo.luminosity_distance(sdss_cat['Z_CIV'][xm_id]).to(u.cm).value
L_lya = np.ones_like(F_lya) * -1

L_lya[mask_dist] = np.log10(F_lya[mask_dist] * 1e-17 * 4 * np.pi * Dl[mask_dist]**2)

L_lya[~np.isfinite(L_lya)] = -1.

In [ ]:
from LAE_selection_method import select_LAEs

nb_min, nb_max = 0, 10
r_min, r_max = 17, 24

paus_cat = select_LAEs(paus_cat, nb_min, nb_max, r_min, r_max,
                       ew0min_lya=30, ewmin_other=100)

zspec = sdss_cat[xm_id]['Z_CIV']
zspec[~mask_dist] = -1

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.scatter(L_lya, paus_cat['L_lya'])

xx = [-100, 100]
ax.plot(xx, xx, ls='--', c='r')

ax.set(xlim=(42, 46), ylim=(42, 46))

plt.show()

In [ ]:
from jpasLAEs.utils import bin_centers
from paus_utils import z_NB

z_min, z_max = z_NB(nb_min), z_NB(nb_max)

zspec_mask = (zspec >= nb_min) & (zspec <= nb_max)

# Completeness SDSS based
L_bins = np.linspace(41, 46, 30)
L_bins_c = bin_centers(L_bins)
h_sdss_qso, _ = np.histogram(L_lya[zspec_mask], L_bins)
h_sel, _ = np.histogram(L_lya[paus_cat['nice_lya'] & zspec_mask], L_bins)

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(L_bins_c, h_sel / h_sdss_qso,
        drawstyle='steps-mid')

plt.show()


###############################################
###############################################

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(L_bins_c, h_sel, label='Selected (Estimated)',
        drawstyle='steps-mid')
ax.plot(L_bins_c, h_sdss_qso, label='All (SDSS)',
        drawstyle='steps-mid')

ax.legend(fontsize=11)

plt.show()

In [ ]:
sdss_Flya_all = sdss_line_cat['LINEAREA'] * 1e-17
sdss_Dl_arr = cosmo.luminosity_distance(sdss_line_cat['LINEZ']).to(u.cm).value
sdss_L_lya_all = np.log10(sdss_Flya_all * 4 * np.pi * sdss_Dl_arr**2)


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.hist(sdss_L_lya_all, L_bins,
        histtype='step', density=True,
        label='All (SDSS)')
ax.hist(L_lya, L_bins, label='X-match (SDSS)',
        histtype='step', density=True)

ax.set_ylabel('\# sources [A. U.]')
ax.set_xlabel(r'$\log_{10}[L_{\mathrm{Ly}\alpha}$ / erg\,s$^{-1}$\,cm$^{-2}$\AA$^{-1}$]')

ax.legend(fontsize=12)

plt.show()